# <div align="center">Instituto Superior de Engenharia de Lisboa</div>
## <div align="center">Licenciatura em Engenharia Informática e Multimédia</div>
<br>
<div align="center">
  <img src="images/isel.png" alt="ISEL Logo" width="300">
</div>

## <div align="center"><b>Processamento de Imagem e Visão</b></div>

### <div align="center"><h1>Trabalho Prático 2B - Deteção de Movimento</h1></div>

<br>
<div align="center">
  <b>Alunos:</b><br>
  A45885 André Silva<br>
  A47094 Pedro Azevedo <br><br>
  
  <b>Docente:</b>
  Pedro Jorge
</div>


# Índice
1. [Introdução](#introduction)
2. [Estimação de imagem de fundo](#estimacao)
3. [Deteção de pixeis ativos](#deteccao)
4. [Utilização de operadores morfológicos](#operadores)
5. [Deteção de regiões ativas](#deteccao)
6. [Vizualiçação dos resultados do processamento](#visualizacao)
7. [Conclusões](#conclusao)
8. [Bibliografia](#bibliografia)



### 1. Introdução <a id="introducao"></a>

No âmbito da unidade curricular de Processamento de Imagem e Visão, foi proposto o desenvolvimento de um algoritmo de visão de computador que permita detetar, classificar, zonas da imagem onde ocorreram movimentos de objetos. Para tal, foi disponibilizado um video, que serve de base para o desenvolvimento do algoritmo. Adicionalmente, foi lecionado nas aulas teóricas e práticas, um conjunto de técnicas e algoritmos que permitem a resolução deste problema. Como a estimação de imagem de fundo, deteção de pixeis ativos, a utilização de operadores morfológicos e de regióes ativas e por fim a classificação das mesmas. Assim, para o desenvolvimento do trabalho, foi utilizado a linguagem de programação Python e a biblioteca OpenCV, que permite a utilização de algoritmos de visão de computador.

Neste relatório realizado em Jupyter Notebook, é apresentado o desenvolvimento do algoritmo, bem como a sua explicação e justificação. Para tal, o relatório está dividido em 5 secções, que correspondem às 5 fases de desenvolvimento do algoritmo. Cada secção é composta por uma breve explicação do algoritmo desenvolvido, acompanhado pela sua implementação em Python. No final do relatório, é apresentado o video final, que contém o resultado do processamento do video base, bem como uma breve conclusão do trabalho realizado.



Sequencia de tarefas:
- Estimação de imagem de fundo (Sugestão: usar a filtragem temporal com filtrode mediana)
- Deteção de pixeis ativos.
- Utilização de operadores morfológicos
- Deteção de regiões ativas
- Classificação e correspondência de regiões ativas (Sugestão para a correspondência: "People tracking in surveillance applications", Luis M.Fuentes & Sergio A. Velastin, Proceedings 2nd IEEE Int. Workshop on PETS, Kauai, Hawaii, USA, 2001)
- Vizualiçação dos resultados do processamento.

In [240]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

### 2 - Estimação de imagem de fundo <a id="estimacao"></a>

Primeiro passo para detetar movimento é estimar a imagem de fundo, para que, frame a frame se compare com o fundo estimado detetar movimento.
Para estimar o fundo, é necessário fazer a mediana temporal, ou seja, calcular a mediana de cada pixel ao longo do tempo, dos frames selecionados. Como se retirasse os objetos em movimento, e ficasse apenas com o fundo. Como ao longo do tempo o fundo alterasse, ou existe mudança de luz, ou no caso do video de teste, existem carros estacionados que saem e outros que estacionam, é necessário, ao longo do processamento dos frames, ir recalculando o fundo estimado. Ao longo do processamento, escolheu-se estimar o fundo a cada 300 frames, ou seja, a cada 300 frames, é recalculado o fundo estimado, com os 300 frames anteriores, adicioando os frames a lista "frames_estimar_fundo", utilizando o metodo "read()", que lê o frame seguinte do video.

In [241]:
def estimar_fundo(frames_estimar_fundo, frame):
 # Estimar o fundo
    if frames_estimar_fundo:
        return np.median(frames_estimar_fundo, axis=0).astype(np.uint8)
    else:
        # Lida com o caso em que a lista está vazia (nenhum frame disponível)
       return np.zeros_like(frame)

### 3 - Deteção de pixeis ativos <a id="deteccao"></a>:

Para detetar pixeis ativos, é necessário comparar o frame com o fundo estimado, e aplicar um limiar para detetar pixeis ativos, subtraindo pixel a pixel, e comparando com um limiar, se for maior, é pixel ativo, se for menor, é pixel inativo.
Antes de fazer a diferença entre o frame e o fundo estimado, é necessário decidir se se vai utilizar as imagens a cor ou a escala de cinzento. Para este trabalho, optou-se por utilizar a cores, embora tenha uma complexidade computacional maior, e ocupe mais espaço em memória e armazenamento, tem a capacidade de discriminar melhor objetos semelhantes, e tem mais informação de cor, que pode ser necessária para a classificação das regiões ativas.

Para calcular a diferença, utilizou-se a função "cv.absdiff()", onde se passa como argumentos o frame e o fundo estimado, e retorna a diferença absoluta entre os dois. Para comparar com o limiar, é necessário comverter a diferença em escla de cinzento, utilizando a função "cv.cvtColor()". Para aplicar o limiar, utilizou-se a função "cv.threshold()", onde se passa como argumentos a diferença, o limiar, o valor máximo, e o tipo de limiarização, neste caso, binário. O valor máximo é 255, pois é o valor máximo de um pixel em escala de cinza, e o tipo de limiarização é binário, pois queremos apenas dois valores, 0 ou 255, ou seja, preto ou branco. Caso o valor do pixel seja maior que o limiar, é atribuído o valor 255, caso seja menor ou igual, é atribuído o valor 0, tendo no final uma imagem bonaria onde os pixeis ativos são brancos, e os pixeis inativos são pretos.

In [242]:
def pixels_ativos(frame, fundoEstimado, limiar=10):
    
    # Calcula a diferença absoluta entre o frame e o fundo estimado
    diferenca = cv.absdiff(frame, fundoEstimado)
    
    # Converte a diferença para monocromático para aplocar um limiar
    diferenca_gray = cv.cvtColor(diferenca, cv.COLOR_BGR2GRAY)
    
    # Aplica um limiar para identificar os pixels ativos
    _, pixels_ativos = cv.threshold(diferenca_gray, limiar, 255, cv.THRESH_BINARY)
    
    return pixels_ativos

### 4 - Utilização de operadores morfológicos <a id="operadores"></a>: 

Ao identificar as regioes ativas, estas não vão ser perfeitas, e vão ter ruído, para isso, é necessário aplicar operadores morfológicos, para eliminar o ruído, e melhorar a qualidade das regiões ativas. Aplicou-se então os operadores de fechamento, erosão e dilatação, para eliminar ruido e melhorar a qualidade das regiões ativas. Para tal, utilizou-se as funções "cv.getStructuringElement()", "cv.morphologyEx()", "cv.erode()" e "cv.dilate()". Para o operador de fechamento, utilizou-se um kernel retangular de 5x5, para a erosão, utilizou-se um kernel elíptico de 5x5, e para a dilatação, utilizou-se um kernel cruz de 3x8. Alterando os parametros dos kernels, ou das interações, podemos obter resultados diferentes, sendo que esta definição de kernels e interações, foi uma que obteve bons resultados.

In [243]:
def operadores_morfologicos(frame):

    # Aplicar operação de fechamento
    kernel_close = cv.getStructuringElement(cv.MORPH_RECT, (5, 5))
    nframe = cv.morphologyEx(frame, cv.MORPH_CLOSE, kernel_close)


    kernel = cv.getStructuringElement(cv.MORPH_ELLIPSE, (5, 5))
    nframe = cv.erode(nframe, kernel, iterations=2)

    kernel = cv.getStructuringElement(cv.MORPH_CROSS, (3, 8))
    nframe = cv.dilate(nframe, kernel, iterations=5)

    return nframe

### 5 - Classificação das regiões ativas <a id="classificacao"></a>:

Tendo as regiões ativas, falta identifica-as, extrair as propriedades e classifica-las. 
Aplicando o "findCountours()", é possível identificar os contornos das regiões ativas, e com o "boundingRect()", consegue-se obter o retângulo que envolve o contorno. 
Através das propriedades do retângulo de cada contorno, é possível extrair as caracteristicas da região, como a área, o aspect ratio, a posição vertical, o histograma de cores e o centro, aplicando certas formulass matemáticas e do OpenCV:

Area - cv.contourArea(contorno)
Aspect Ratio - Largura / Altura
Posição Vertical - Posição Y / Numero de linhas da imagem
Histograma de Cores - media das cores do retângulo, selecionando apenas as 3 componentes de cores RGB, "cv.mean(frame_original[y:y + h, x:x + w])[:3]"
Centro - Posição X + Largura / 2, Posição Y + Altura / 2

Com estas propriedades, é possível classificar a região, comparando os valores das propriedades com certos limiares definidos, e aplicando a seguinte lógica:

Se o aspect ratio for maior que o limiar de aspect ratio de carro, e a posição vertical for menor que o limiar de posição vertical, a região é um carro.
Se o aspect ratio for menor que o limiar de aspect ratio de pessoa, e a posição vertical for maior que o limiar de posição vertical de pessoa, a região é uma pessoa.
Se não for nenhuma das anteriores, a região é outra.

Os limiares foram definidos experimentalmente, analisando o resultado e ajustando os valores consuante o resultado obtido, tendo estes os valores dos limiares:
Limiar de aspect ratio de carro: 0.50
Limiar de aspect ratio de pessoa: 0.60
Limiar de posição vertical: 0.80
Limiar de posição vertical de pessoa: 0.50

Se houver alteração nos operadores morfológicos, estes parametros têm que ser ajustados, pois o tamanho das regiões vai ser diferente.

Por fim, é necessário desenhar o retângulo e o resultado da classificação da região ativa, para isso, utilizou-se as funções "cv.drawContours()", sendo que para cada categoria, é atrbuida uma cor diferente, e "cv.putText()", onde se escreve a categoria, o aspect ratio e a posição vertical, no retângulo da região ativa.


In [244]:
def active_region(frame_f, frame_bin, min_width, min_height):
    contours, _ = cv.findContours(frame_bin, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)

    # Lista para armazenar os contours que correspondem a regiões ativas neste frame
    contours_atuais = []

    for (i, c) in enumerate(contours):
        (x, y, w, h) = cv.boundingRect(c)  # Obtém as coordenadas do retângulo

        valid_contour = (w >= min_width) and (h >= min_height)  # Verifica se o retângulo é válido
        if not valid_contour:
            continue

        # Extrai características da região
        propriedades = extrair_caracteristicas_regiao(c, x, y, w, h, frame_f)
        area, aspect_ratio, posicao_vertical, histograma_cores, centro = propriedades
        # Classifica a região com base nas características
        categoria = classificar_regiao(c, propriedades)

        # Adiciona o contour atual à lista
        contours_atuais.append(c)

        # Desenha o retângulo colorido de acordo com a categoria
        cor_retangulo = (0, 255, 0) if categoria == "Pessoa" else (255, 0, 0) if categoria == "Carro" else (0, 0, 255)
        cv.rectangle(frame_f, (x, y), (x + w, y + h), cor_retangulo, 2)

        # Escreve as propriedades e a label

        cv.putText(frame_f, "AR: {:.2f}".format(aspect_ratio), (x, y - 5), cv.FONT_HERSHEY_SIMPLEX, 0.5, cor_retangulo, 2)
        cv.putText(frame_f, "PV: {:.2f}".format(posicao_vertical), (x, y + h + 15), cv.FONT_HERSHEY_SIMPLEX, 0.5, cor_retangulo, 2)
        cv.putText(frame_f, f"{categoria}", (x, y + h + 30), cv.FONT_HERSHEY_SIMPLEX, 0.5, cor_retangulo, 2)

    return frame_f


In [245]:
def extrair_caracteristicas_regiao(contorno, x, y, w, h, frame_original):
    area = cv.contourArea(contorno)
    aspect_ratio = w / h if h != 0 else 0
    posicao_vertical = y / frame_original.shape[0]
    histograma_cores = cv.mean(frame_original[y:y + h, x:x + w])[:3]
    centro = (x + w // 2, y + h // 2)


    return area, aspect_ratio, posicao_vertical, histograma_cores, centro

In [246]:
def classificar_regiao(contorno, propriedades):
    
    area, aspect_ratio, posicao_vertical, histograma_cores, _ = propriedades
    limiar_aspect_ratio_carro = 0.50  # Limiar para aspect ratio de carro (0.38 - 1.n) (pessoa < .50)
    limiar_aspect_ratio_pessoa = 0.60  # Limiar para aspect ratio de pessoa 
    limiar_posicao_vertical = 0.80  # Limiar para posição vertical (carro 0.56) (pessoa: 0.80 > 506
    limiar_posicao_vertical_pessoa = 0.50  # Limiar para posição vertical de pessoa (carro 0.56) (pessoa: 0.80 > 506

    # Lógica de classificação
    if aspect_ratio >= limiar_aspect_ratio_carro and posicao_vertical <= limiar_posicao_vertical:
        categoria = "Carro"
    elif aspect_ratio <= limiar_aspect_ratio_pessoa and posicao_vertical >= limiar_posicao_vertical_pessoa:
        categoria = "Pessoa"
    else:
        categoria = "Outro"

    
    return categoria

### 6 - Vizualiçação dos resultados do processamento <a id="visualizacao"></a>:

Nesta função, começa-se por ler o video, e definir o número de frames que vão ser utilizados para estimar o fundo, neste caso, 300. De seguida, é necessário definir o número total de frames do video, para que o processamento pare quando chegar ao fim do video. 
De seguida, é necessário definir as configurações para guardar o video final, como o fps, a largura e altura, e o codec.

Para começar o processamento do video, verifica-se se o número de frames que faltam processar, estão dentro do numero de frames que vão ser utilizados para estimar o fundo, se não estiverem, o número de frames que vão ser utilizados para estimar o fundo, é igual ao número de frames que faltam processar. De seguida, é necessário iniciar as listas de frames que vão ser utilizados para estimar o fundo, e os frames que vão ser processados. Estima-se o fundo, e percorre-se a lista de frames a serem processados, aplicando as funções anteriores, e adicionando o resultado à lista de frames finais, incrementando também o número de frames processados. Por fim, escreve-se o video final, e quando o processamento termina, fecha-se o video e o video final.

In [247]:
def processar_video(video_path):
    vid = cv.VideoCapture(video_path)
    nFramesFundo = 300
    nFramesProcessados = 0
    nTotalFrames = int(vid.get(cv.CAP_PROP_FRAME_COUNT))
    framesFinais = []


    # Configuração para salvar o vídeo final
    fps = int(vid.get(cv.CAP_PROP_FPS))
    width = int(vid.get(cv.CAP_PROP_FRAME_WIDTH))
    height = int(vid.get(cv.CAP_PROP_FRAME_HEIGHT))
    fourcc = cv.VideoWriter_fourcc(*'XVID')
    output_video = cv.VideoWriter('output_video.avi', fourcc, fps, (width, height))

    try:
        # Enquanto houver vídeo
        while nFramesProcessados < nTotalFrames:
            if nTotalFrames - nFramesProcessados < nFramesFundo:
                nFramesFundo = nTotalFrames - nFramesProcessados
    
            # Iniciar a lista de frames que vão estimar o fundo
            frames_estimar_fundo = []
            # Iniciar a lista de frames que vão ser processados
            framesPorProcessar = []
    
            for i in range(nFramesFundo):
                ret, frame = vid.read()
                if not ret:
                    break
    
                # Adicionar à lista dos frames que vão estimar o fundo
                frames_estimar_fundo.append(frame)
                # Adicionar à lista dos frames que vão ser processados
                framesPorProcessar.append(frame)
            
            fundo_estimado = estimar_fundo(frames_estimar_fundo, frame)
    
           
    
            # Processar os primeiros 500 frames
            for frame in framesPorProcessar:
                # Calcular os pixels ativos
                frame_bin = pixels_ativos(frame, fundo_estimado, 50)
                # Aplicar operadores morfológicos
                frame_operadores = operadores_morfologicos(frame_bin)
                # Processar as regiões ativas e obter o novo mapa de regiões ativas
                frame_final= active_region(frame, frame_operadores, 5, 5)
                framesFinais.append(frame_final)
                nFramesProcessados += 1
                # Escrever o vídeo final
                output_video.write(frame_final)

    except Exception as e:
        print(f"Ocorreu um erro: {e}")

    finally:
        vid.release()
        output_video.release()
        cv.destroyAllWindows()


In [248]:
processar_video("camera1.avi")

### 7 - Conclusão <a id="conclusao"></a>:

Após o procecssamento do video, conseguimos concluir que o algoritmo desenvolvido, consegue detetar e classificar as regiões ativas, tem certas falhas em relação a deteção das regiões, onde estas ficam mais pequenas, não conseguindo identificar como qualquer objeto, ou ficar classificar como outro objeto. Para chegar a a um melhor resultado, teria que se ajustar os parametros dos operadores morfologicos.
Para melhorar o algoritmo, poderia-se fazer a correspondencia entre regiões, através da propriedade do centro de cada região e medir a distancia euclidiana entre os centros do frame anterior.
 

### 8 - Bibliografia <a id="bibliografia"></a>:

- https://docs.opencv.org/4.x/d2/d96/tutorial_py_table_of_contents_imgproc.html
- Acetatos Capítulo 7 - Análise de Movimento, Professor Arnaldo Abrantes
